# Labels | Dropwise Count

In [1]:
import pandas as pd
import re
from pathlib import Path
import os

In [2]:
OUTPUT_DIR = Path("C:/Playground/bhaiya-orders/outputs/")

In [3]:
from ipyfilechooser import FileChooser

In [4]:
fc = FileChooser("C:/Playground/bhaiya-orders/inputs")
fc.use_dir_icons = True
display(fc)

FileChooser(path='C:/Playground/bhaiya-orders/inputs', filename='', show_hidden='False')

In [5]:
FILE = Path(fc.selected)

In [6]:
try:
    if 'csv' in FILE.suffix: df = pd.read_csv(FILE)
    else: df = pd.read_excel(FILE)
except OSError:
    print("Error occured, invalid file!")

In [7]:
df.head(2)

,Submission Date,Order Number#,Token,First Name,Last Name,Select Drop Point and Building,Select Wing,Flat No.,Phone Number,Email,Delivery Date,Your Order: Products
0,2020-07-21 08:27:15,BHORS-0557,1,Rashmita,Sundaram,Aeon/Zeon - Aeon,A,1202,9004913659,rashmita.sundaram@gmail.com,Wed 22nd July,Sitaphal/Custard Apple Per KG (Amount: 130.00 ...
1,2020-07-20 21:53:06,BHORS-0548,2,Heena,Parekh,Aeon/Zeon - Aeon,A,1101,9892404976,heenaparekh75@gmail.com,Wed 22nd July,"Muskmelon Per Pc (Amount: 110.00 INR, Special ..."


# Manual (Obsolete)

This section creates labels from the Pipe Separated File, which was done manually earlier.

In [45]:
# ITEMS = 'Your Order: Products'

In [46]:
# df[ITEMS][0]

In [47]:
# cols = df.columns

In [48]:
# # splitting on pipe
# new_df = pd.DataFrame(columns=df.columns)

# for index, row in df.iterrows():
#     row_dict = {}
    
#     for col in cols:
#         row_dict[col] = row[col]
        
#     spl = row[ITEMS].split('|')
#     for item in spl:
#         item = item.strip()
#         if len(item) > 2:
#             row_dict[ITEMS] = item
#             new_df = new_df.append(row_dict, ignore_index=True)

In [49]:
# new_df.to_csv('separate-pipe.csv', index=False)

In [50]:
# # replicate count number of times
# brand_new_df = pd.DataFrame(columns=new_df.columns)

# for index, row in new_df.iterrows():
#     spl = row[ITEMS].split('-')
#     for i in range(int(spl[-1])):
#         brand_new_df = brand_new_df.append(row, ignore_index=True)

# brand_new_df[ITEMS] = brand_new_df[ITEMS].apply(lambda x: x.split('-')[0])

In [51]:
# brand_new_df.to_csv('separate-pipe-make-labels.csv', index=False)

# Use This

In [8]:
# df["Token"] = list(range(len(df)))

In [9]:
for col in df.columns:
    if "Your Order" in col:
        your_order = col
        
    if "token" in col.lower():
        token = col
        
    if "flat" in col.lower():
        flat = col
        
    if "order number" in col.lower():
        order_number = col

In [10]:
df[your_order][0]

'Sitaphal/Custard Apple Per KG (Amount: 130.00 INR, Special Quantity: 1)\nElaichi Banana One Dozen (Amount: 80.00 INR, Special Quantity: 1)\nSubtotal: \nTax: \n\nTotal: 210.00'

In [11]:
def get_item_name(order):
    '''
    Return the Item Name, Final Weight, Num Packets, Unit
    '''
    
    stopwords = ['per', 'gms', 'gm', 'kgs', 'kg', 'pack', 'each', '-']
    
    # parse the key, stop when you encounter a number, unit, keyword like 'per', 'pack'
    item = order.split('(')[0].strip()
    item = re.sub('[0-9]+', ' ', item)
    item = item.split()
#     print(item)
    final_item = ''
    for token in item:
        if token.lower() in stopwords:
            break
        final_item += ' ' + token
    
    return final_item.strip()


def get_qty(txt):
    
    """
        Returns the Special Quantity key for a given order. 
            Sample input string: '...no. of 500 gm packets required: 1)'
            Returns: 1
    """
    
    if txt[-1] != ')':  txt = txt + ')'
    
    match = re.search(": ([0-9]+?)\)", txt)

    # temporary fix for VNR Guava
    if "VNR Guava Per Kg (" in txt: match = re.search("\(([0-9]+?)", txt)
    
    if match:
        return int(match.group(1))
    
    
def extract_num(txt):
    
    """
        Searches a string and returns the last number found, if present. 
        Otherwise return 1.
        Use case: For an item key such as 'Tondli 300 gms ', return 300 (to calculate total qty required)
    """
    
    match = re.findall("([0-9]+)", txt)
    if match:
        return int(match[-1])
    else:
        return 1
    
    
    
def get_unit(txt):
    """
        Gets unit from an item string.
        Assumption: standard occurence of units all across the board. 
    """
    txt = txt.lower()
    if txt.find(' gm') != -1: unit = 'gms'
    elif (txt.find(' pc') != -1) or (txt.find(' pack') != -1): unit = 'pcs'
    elif txt.find(' bunch') != -1: unit = 'bunches'
    else: unit = 'kgs'
    return unit

In [12]:
def parse_order(order):
    '''
    Return the Item Name, Final Weight, Num Packets, Unit
    '''
    if len(order.split('(')) < 2: return None
    
    item_name = get_item_name(order)
    num_packets = get_qty(order)
    unit = get_unit(order)
    qty_per_pack = extract_num(order.split('(')[0])
    final_weight = qty_per_pack * num_packets

    return {"item_name": item_name,
            "num_packets": num_packets,
            "unit": unit,
            "final_weight": final_weight}

In [13]:
rows_list = []
for index, row in df.iterrows():
    for order in row[your_order].split(')'):
        parsed = parse_order(order)
        if parsed:
            for q in range(parsed['num_packets']):
                row_dict = {}
                row_dict['token_no'] = row[token]
                row_dict['order_no'] = row[order_number]
                row_dict['first_name'] = row["First Name"]
                row_dict['last_name'] = row["Last Name"]
                row_dict['item_name'] = parsed['item_name']
                row_dict['drop_point'] = row['Select Drop Point and Building'].split('-')[0]

    #             # TEMP
    #             row_dict['final_weight'] = parsed['final_weight']
    #             row_dict['num_packets'] = parsed['num_packets']

                row_dict['building'] = row['Select Drop Point and Building']
                row_dict['wing'] = row['Select Wing']
                row_dict['flat_no'] = row[flat]
                row_dict['phone_number'] = row['Phone Number']
                row_dict['email_id'] = row['Email']

    #             if 'tondli' in row_dict['item_name'].lower() or 'palak' in row_dict['item_name'].lower() or 'raw banana' in row_dict['item_name'].lower():
                rows_list.append(row_dict)

In [14]:
new_df = pd.DataFrame(rows_list)

In [15]:
new_df.tail()

,token_no,order_no,first_name,last_name,item_name,drop_point,building,wing,flat_no,phone_number,email_id
243,74,BHORS-0545,Deepti,Sanghi,Kiwi,Sector 1/2,Sector 1/2 - Silver Arch,A,501,9819336770,sanghideepti@yahoo.in
244,74,BHORS-0545,Deepti,Sanghi,Papaya,Sector 1/2,Sector 1/2 - Silver Arch,A,501,9819336770,sanghideepti@yahoo.in
245,74,BHORS-0545,Deepti,Sanghi,Papaya,Sector 1/2,Sector 1/2 - Silver Arch,A,501,9819336770,sanghideepti@yahoo.in
246,75,BHORS-0574,Rohan,Goel,Chausa Mangoes,Sector 1/2,Sector 1/2 - Topaz,NaN,202,9619935777,rohangoel68@gmail.com
247,76,BHORS-0558,Mithila,Naik,Sitaphal/Custard Apple,Sector 1/2,Sector 1/2 - Topaz,NaN,601,9870851416,mithila_lad@yahoo.com


In [16]:
save_pth = OUTPUT_DIR/FILE.stem
os.makedirs(save_pth, exist_ok=True)

### Save DropWise

In [17]:
fname = save_pth/('dropwise-'+ str(FILE.stem) + '.csv')

In [18]:
new_df.groupby(['drop_point', 'item_name']).count().token_no.to_csv(fname, header=['count'])

### Save Labels

In [17]:
fname = save_pth/('labels-' + str(FILE.stem) + '.csv')

In [18]:
new_df.to_csv(fname, index=False)